# Description

This python script performs continuum removal (CR) on spectra of ternary mixtures. \
In addition, on each isolated band it computes band area (BA), band depth (BD) and band centre (BC). 

In [1]:
# import

import numpy as np #numeric python
import matplotlib.pyplot as plt #plots
import matplotlib.ticker as ticker #set ticks spacing 
import seaborn as sns

#function to find nearest value in a numpy array 
from find_nearest import find_nearest

%matplotlib

Using matplotlib backend: <object object at 0x10928b360>


# Read and plot data

In [2]:
#reading data

path = 'Data/'
endmember1_name = 'NAu-2' #choose among SM1200H, NAu-1, NAu-2
endmember2_name = 'HEX'
endmember3_name = 'FV7'

#assuming clay+'-'+x_ls[i]+'_'+hex+'-'+y_ls[i]+'_'+basalt+'-'+z_ls[i]+'_0000n.asd.rts.txt filename: 

x_ls = ['80', '70', '60', '50','40', '30', '20', '10',
        '70', '60', '50', '40', '30', '20', '10',     
        '60', '50', '40', '30','20', '10',   
        '50', '40', '30', '20','10',  
        '40', '30', '20', '10',  
        '20', '10'] # % clay

y_ls = ['10', '20', '30', '40','50', '60', '70', '80',
        '10', '20', '30', '40','50', '60', '70',
        '10', '20', '30', '40','50', '60',
        '10', '20', '30', '40','50',
        '10', '20', '30', '40',
        '10', '20'] # % hex

z_ls = ['10', '10', '10', '10','10', '10', '10', '10',
        '20', '20', '20', '20', '20', '20', '20',     
        '30', '30', '30', '30','30', '30',   
        '40', '40', '40', '40','40',  
        '50', '50', '50', '50',  
        '70', '70'] # % basalt


means_ls = [] #list containing mean spectra 
labels_ls = [] #list with labels

for i in range(0, len(x_ls)):

    s0 = np.loadtxt(path+endmember1_name+'-'+x_ls[i] + '_'+endmember2_name+'-'+y_ls[i] + '_'+endmember3_name+'-'+z_ls[i]+'_00000.asd.rts.txt', delimiter='\t', skiprows = 1)
    s1 = np.loadtxt(path+endmember1_name+'-'+x_ls[i] + '_'+endmember2_name+'-'+y_ls[i] + '_'+endmember3_name+'-'+z_ls[i]+'_00001.asd.rts.txt', delimiter='\t', skiprows = 1)
    s2 = np.loadtxt(path+endmember1_name+'-'+x_ls[i] + '_'+endmember2_name+'-'+y_ls[i] + '_'+endmember3_name+'-'+z_ls[i]+'_00002.asd.rts.txt', delimiter='\t', skiprows = 1)

    s_mean = (s0[:,1]+s1[:,1]+s2[:,1])/3

    means_ls.append(s_mean)
    labels_ls.append(endmember1_name+'-'+x_ls[i]+'_'+endmember2_name+'-'+y_ls[i]+'_'+endmember3_name+'-'+z_ls[i])

wvl = s0[:,0]*0.001 #wavelength in microns
spectra = np.array(means_ls).T #all the spectra go from the list to this numpy array
print('Spectra shape:',spectra.shape) #checking shape

Spectra shape: (2151, 32)


In [ ]:
%matplotlib
#plot with or without offset
fig_s, axs_s = plt.subplots(figsize=(10,8))

#palette
palette = sns.color_palette("viridis", len(labels_ls)).as_hex() #https://matplotlib.org/stable/tutorials/colors/colormaps.html
axs_s.set_prop_cycle(color=palette)

#offset: comment line with off = 0 if you need to use an offset
off = np.arange(start = 0, stop = len(spectra[0,:])) * 0.02
off = 0

#plotting spectra
axs_s.plot(wvl, spectra + off, label = labels_ls)

#y, x label
axs_s.set_ylabel('Reflectance', fontsize=10)
axs_s.set_xlabel('Wavelength ('+r'$\mu$m)', fontsize=10)

#x tick spacing
tick_spacing_major = 0.2
tick_spacing_minor = 0.1
axs_s.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing_major)) #big ticks
axs_s.xaxis.set_minor_locator(ticker.MultipleLocator(tick_spacing_minor)) #secondary ticks

plt.legend(bbox_to_anchor=(1.3,0.5), loc='center', ncol=1, reverse = True) #show legend next to the plot, in 1 col with reversed labels
plt.tight_layout()
plt.show()


Using matplotlib backend: MacOSX


# Smoothing (boxcar) - optional

In [34]:
#IDL smoothing
from IDL_smooth import smooth
#smooth(data, w) -> data = array, w = window size (must be odd); e.g. w = 3

spectra_smoothed = np.zeros_like(spectra)

for i in range (0,32):
    spectra_smoothed[:,i] = smooth(spectra[:,i],13)

In [ ]:
%matplotlib
#plot with or without offset
fig_s, axs_s = plt.subplots(figsize=(10,6))

#palette
palette = sns.color_palette("viridis", len(labels_ls)).as_hex() #https://matplotlib.org/stable/tutorials/colors/colormaps.html
axs_s.set_prop_cycle(color=palette)

#offset: comment line with off = 0 if you need to use an offset
off = np.arange(start = 0, stop = len(spectra[0,:])) * 0.02 
off = 0

#plotting spectra
axs_s.plot(wvl, spectra_smoothed + off, label = labels_ls)

#y, x label
axs_s.set_ylabel('Reflectance', fontsize=10)
axs_s.set_xlabel('Wavelength ('+r'$\mu$m)', fontsize=10)

#x tick spacing
tick_spacing_major = 0.2
tick_spacing_minor = 0.1
axs_s.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing_major)) #big ticks
axs_s.xaxis.set_minor_locator(ticker.MultipleLocator(tick_spacing_minor)) #secondary ticks

plt.title('Spectra, smoothed')
plt.tight_layout()
#plt.legend(bbox_to_anchor=(1.12,0.5), loc='center', ncol=1, reverse = True) #show legend next to the plot, in 1 col with reversed labels
plt.show()

Using matplotlib backend: MacOSX


In [ ]:
#---- WARNING ----#
# only run this cell if you want to compute the spectral parameters on smoothed spectra
spectra = spectra_smoothed 

# Continuum removal

## Function definition 

In [16]:
def find_line (left_sh, right_sh, axs):

    '''input: 
    left_sh = left shoulder position (in microns)
    right_sh = right shoulder position (in microns)
    axs = ax variable to overplot continuum removal line on the spectra

    output: 
    m = angular coefficient of the line (coefficiente angolare)
    q = y-intercept (intercetta/termine noto)
    axs 
    '''

    ls_index = find_nearest(left_sh, wvl) #left shoulder index
    rs_index = find_nearest (right_sh, wvl) #right shoulder index

    YL = spectra[ls_index,:] #left shoulder y coordinate (wvl) for all the spectra 
    YR = spectra[rs_index,:] #right shoulder y coordinate (wvl) for all the spectra 

    length = len(YL) #length = number of spectra 

    XL = np.full(length, wvl[ls_index]) #left shoulder x coordinate (wvl) - (the array is filled all with the same number)
    XR = np.full(length, wvl[rs_index]) #right shoulder x coordinate (wvl) - (the array is filled all with the same number)

    #overplot chosen left and right shoulder position 
    axs.scatter(XL, YL, color='red', s=15) 
    axs.scatter(XR, YR, color='red', s=15)

    #plot a line connecting the shoulders
    axs.plot((XL, XR), (YL, YR), 'r-')
    plt.show()

    #find the equation of the line passing through the shoulders
    #y=mx+q

    m = (YR-YL)/(XR-XL)
    q = YL-m*XL


    return (m,q)


def continuum_removal(left_sh, right_sh, m, q):


    '''input: 
    left_sh = left shoulder position (in microns) - float
    right_sh = right shoulder position (in microns) - float
    m = angular coefficient of the line (coefficiente angolare) - array
    q = y-intercept (intercetta/termine noto) - array

    output: 
    y_cr = continuum removed reflectance values - array

    also plots the continuum removed bands
    
    '''


    #retrieve shoulder indexes
    ls_index = find_nearest(left_sh, wvl) #left shoulder 
    rs_index = find_nearest (right_sh, wvl) #right shoulder 

    #remove the continuum

    Y = np.transpose(spectra[ls_index:rs_index+1,:]) #array containing the original reflectance values, in the selected range, transposed
    X = wvl[ls_index:rs_index+1].reshape(1,-1) #array containing the wavelengths in the selected range, transposed
    M = m[:,np.newaxis] #get a (32,1) array from the original (32,) of m
    Q = q[:,np.newaxis] #get a (32,1) array from the original (32,) of q

 
    y_cr = Y / (M*X+Q)
    
    #plot
    fig, axs = plt.subplots(figsize=(10,6)) #figure and axes
    axs.set_prop_cycle(color=palette) #set color 

    axs.plot(wvl[ls_index:rs_index+1], np.transpose(y_cr), label = labels_ls) #plot
    axs.set_ylabel('Reflectance (continuum removed)')
    axs.set_xlabel('Wavelength ('+r'$\mu$m)')

    plt.show()

    return(np.transpose(y_cr))



## Band-to-band CR

In [17]:
#Ls_arr = np.array([0.84, 1.29, 1.82, 2.25]) #left shoulder's array for SM1200H 
#Rs_arr = np.array([1.197, 1.803, 2.18, 2.346]) #right shoulder's array for SM1200H 

#Ls_arr = np.array([0.427, 0.4718, 0.5887, 0.7666, 1.311, 1.8174, 2.1836, 2.2584]) #left shoulder's array for Nau1
#Rs_arr = np.array([0.4706, 0.5641, 0.7501, 1.297, 1.805, 2.165, 2.2247, 2.3212]) #right shoulder's array for Nau1

Ls_arr = np.array([0.4889, 0.5803, 0.809, 1.314, 1.825, 2.258]) #left shoulder's array for Nau2 
Rs_arr = np.array([0.5494, 0.7713, 1.308, 1.813, 2.2125, 2.3317]) #right shoulder's array for Nau2


cr_spectrum_zeros = np.ones_like(spectra) #creating an array containing the y_cr (cr reflectance) at their respective range of wvl and zeros in all the other positions
y_bc = [] #band centre (y)
ba = [] #band area

#computing BD, BA, BC
for i, value in enumerate(Ls_arr):
    m,q = find_line(Ls_arr[i], Rs_arr[i], axs_s)
    y_cr = continuum_removal(Ls_arr[i], Rs_arr[i], m, q) #cr spectra
    cr_spectrum_zeros[find_nearest(Ls_arr[i], wvl):find_nearest(Rs_arr[i], wvl)+1]=y_cr

    y_bc.append(np.min(y_cr, axis = 0)) #list of array where every array has the minima of the spectra for a specific band
    

    x_cr = wvl[find_nearest(Ls_arr[i], wvl):find_nearest(Rs_arr[i], wvl)+1]
    
    a1 = np.trapz(y_cr, x_cr, axis = 0) #area below the curve
    a2 = np.max(x_cr)-np.min(x_cr) #rectangle of base x_cr and height=1
    ba.append(a2-a1)
    

#turning the list into an array
y_bc = np.asarray(y_bc)
ba = np.asarray(ba) 
bd = 1.0 - y_bc #band depth


In [7]:
x_bc= np.zeros_like(y_bc) #same as np.shape(y_bc) = (n,32); n = number of absorption features

for i in range (0,len(y_bc[:,0])): #n
    for j in range(0,len(y_bc[0,:])): #32
        x_bc[i,j] = wvl[find_nearest(y_bc[i,j], cr_spectrum_zeros[:,j])]

In [10]:
#save in a formatted txt file 

np.savetxt('band_centre_ternary_'+endmember1_name, np.transpose(x_bc), header = 'Columns = band centre values; rows = samples, %Clay/Hex/Basalt\nSample type = '+endmember1_name+'+'+endmember2_name+'+'+endmember3_name+' starting at % 80/10/10 - see labels file', fmt = '%.8f')
np.savetxt('band_depth_ternary_'+endmember1_name, np.transpose(bd), header = 'Columns = band depth values; rows = samples, %Clay/Hex/Basalt\nSample type = '+endmember1_name+'+'+endmember2_name+'+'+endmember3_name+' starting at % 80/10/10 - see labels file ', fmt = '%.8f')
np.savetxt('band_area_ternary_'+endmember1_name, np.transpose(ba), header = 'Columns = band area values, computed with the trapezoidal rule; rows = samples, %Clay/Hex/Basalt\nSample type = '+endmember1_name+'+'+endmember2_name+'+'+endmember3_name+' starting at % 80/10/10 - see labels file', fmt = '%.8f')
#save labels
np.savetxt('labels_ternary_'+endmember1_name, labels_ls, header = 'Label file', fmt = '%s')